### Импортируем необходимые библиотеки

In [21]:
import csv
import operator
import json
from math import sqrt

### Чтение данных из CSV-файла
Удаляются столбцы и строки с соответствующими им названиями

In [22]:
def csvRead(path, outType = "str"):
    with open(path) as csv_file:
        reader = csv.reader(csv_file, delimiter=',', skipinitialspace=True)
        next(reader)
        if (outType == "str"):
            return {rows[0]: rows[1:] for rows in reader}
        elif (outType == "int"):
            return {rows[0]: list(map(int, rows[1:])) for rows in reader}

### Получение названий всех фильмов

In [23]:
def getMovieTitles(path):
    with open(path) as data:
        data_csv_reader = csv.reader(data, delimiter=',', skipinitialspace=True)
        return next(data_csv_reader)[1:]

## $$sim_{u,v}= \frac{\sum_{i=1}^m u_i v_i}{\sqrt{\sum_{i=1}^m u_i^2}\sqrt{\sum_{i=1}^m v_i^2}}$$

In [24]:
def getSim(ratings, input_user):
    sim = {}
    for user_id in ratings:
        if user_id == input_user:
            continue
        user_ratings = ratings[user_id]

        sum_1 = 0
        sum_2 = 0
        sum_3 = 0

        for i in range(len(input_user_ratings)):
            if input_user_ratings[i] == -1 or user_ratings[i] == -1:
                continue
            sum_1 += input_user_ratings[i] * user_ratings[i]
            sum_2 += input_user_ratings[i] ** 2
            sum_3 += user_ratings[i] ** 2

        sim[user_id] = round(sum_1 / sqrt(sum_2) / sqrt(sum_3), 3)
    return sim

In [25]:
def getNearest(sorted_sim, movie_idx, k = 4):
    nearest = []
    counter = 0
    for user in sorted_sim:
        if counter == k:
            break
        if data[user][movie_idx] != -1:
            nearest.append(user)
        counter += 1
    return nearest

**getAvg(user_ratings)** - Возвращает среднюю оценку пользователя по **просмотренным** фильмам.

In [26]:
def getAvg(user_ratings):
    rated = list(filter(lambda x: x != -1, user_ratings))
    return round(sum(rated) / len(rated), 3)

## $$\hat{r}_{ui}=\overline{r}_u+\frac{\sum_{v=1}^k sim_{vu}(r_{vi}-\overline{r}_v)}{\sum_{v=1}^k \mid sim_{vu} \mid}$$

In [27]:
def getRui(movie_idx):
    sum_1 = 0
    sum_2 = 0

    for v in getNearest(sorted_sim, movie_idx):
        sum_1 += sim[v] * (data[v][movie_idx] - getAvg(data[v]))
        sum_2 += abs(sim[v])
    return round(getAvg(input_user_ratings) + sum_1 / sum_2, 3)

### Корректировка для Task 2
В качестве переменной **correct** может выступать любая величина

In [28]:
def getContextCoefficient(movie_idx):
    coefficient = 1
    correct = 0.01
    for v in getNearest(sorted_sim, movie_idx):
        if context_place[v][movie_idx] == 'h':
            coefficient += correct
        else:
            coefficient -= correct

        if context_day[v][movie_idx] == 'Sat' or context_day[v][movie_idx] == 'Sun':
            coefficient += correct
        else:
            coefficient -= correct

    return coefficient

### Task 2
Для рекомендации фильма пользователю **дома** в **выходной день** был применен подход аналогична первому заданию, <br>
только берутся те фильмы, которые **не были просмотрены** данным пользователем и были просмотрены в **выходные дни** **дома** другими пользователями

In [29]:
def getRecommendations():
    task_1 = {}
    task_2 = {}

    for movie_idx in range(len(input_user_ratings)):
        if input_user_ratings[movie_idx] != -1:
            continue

        task_1[movies[movie_idx]] = getRui(movie_idx)
        task_2[movies[movie_idx]] = round(task_1[movies[movie_idx]] * getContextCoefficient(movie_idx), 3)

    sorted_movies = list(sorted(task_2.items(), key=lambda kv: kv[1], reverse=True))[0]
    return {'User': input_user, '1': task_1, '2': {sorted_movies[0]: sorted_movies[1]}}

### Создание JSON файла

In [33]:
def getJson(rec):
    with open(f'result.json', 'w') as outfile:
        json.dump(rec, outfile, indent=4)
    print("JSON created!")

Основной вход программы.

In [35]:
data = csvRead('src/data.csv', 'int')
context_day = csvRead('src/context_day.csv')
context_place = csvRead('src/context_place.csv')
movies = getMovieTitles('src/data.csv')

input_user_id = input('enter user id: ')
input_user = f'User {input_user_id}'
input_user_ratings = data[input_user]

sim = getSim(data, input_user)
sorted_sim = list(map(lambda x: x[0], sorted(sim.items(), key=lambda kv: kv[1], reverse=True)))

rec = getRecommendations()
print(rec)
getJson(rec)

enter user id: 5
{'User': 'User 5', '1': {'Movie 13': 4.521, 'Movie 24': 3.228, 'Movie 28': 2.934, 'Movie 30': 3.875}, '2': {'Movie 13': 4.431}}
JSON created!
